# Setup

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

print("\n################################################################################\n")

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Wed Nov 23 17:17:04 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## System Setup

In [2]:
# Transformers installation
!pip install transformers datasets

# The Evaluate library provides a simple accuracy function
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 12.4 MB/s 
     |████████████████████████████████| 451 kB 45.9 MB/s 
     |████████████████████████████████| 7.6 MB 66.5 MB/s 
     |████████████████████████████████| 182 kB 74.4 MB/s 
     |████████████████████████████████| 212 kB 64.2 MB/s 
     |████████████████████████████████| 115 kB 70.7 MB/s 
     |████████████████████████████████| 127 kB 69.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 1.4 MB/s 


## Other

In [3]:
import numpy as np
import evaluate
import torch
from datasets import load_dataset
from torch.optim import AdamW
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import get_linear_schedule_with_warmup

In [4]:
TOKEN_MAX_LENGTH = 256
BATCH_SIZE = 8
EPOCHS = 3
LEARNING_RATE = 5e-5
SEED_VALUE = 42
TEST_DATASET_SIZE = 1000
EVALUATE_AFTER_EPOCH = True

In [5]:
np.random.seed(SEED_VALUE)
torch.manual_seed(SEED_VALUE)
torch.cuda.manual_seed(SEED_VALUE)

# Dataset Preparation

In [6]:
# IMDB dataset from https://huggingface.co/datasets/imdb
dataset = load_dataset("imdb")

dataset["train"][100]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

{'text': "Terrible movie. Nuff Said.<br /><br />These Lines are Just Filler. The movie was bad. Why I have to expand on that I don't know. This is already a waste of my time. I just wanted to warn others. Avoid this movie. The acting sucks and the writing is just moronic. Bad in every way. The only nice thing about the movie are Deniz Akkaya's breasts. Even that was ruined though by a terrible and unneeded rape scene. The movie is a poorly contrived and totally unbelievable piece of garbage.<br /><br />OK now I am just going to rag on IMDb for this stupid rule of 10 lines of text minimum. First I waste my time watching this offal. Then feeling compelled to warn others I create an account with IMDb only to discover that I have to write a friggen essay on the film just to express how bad I think it is. Totally unnecessary.",
 'label': 0}

In [7]:
# Separate dataset
train_raw = dataset["train"].shuffle(seed=SEED_VALUE)
train_d_raw = train_raw.select(range(12500))
train_daux_raw = train_raw.select(range(12500, 25000))

test_raw = dataset["test"].shuffle(seed=SEED_VALUE)

In [8]:
# # Subsets - uncomment which set to use

# Full D dataset
train_dataset = train_d_raw
eval_dataset = test_raw

# # Full D_aux dataset
# train_dataset = train_daux_raw
# eval_dataset = test_raw

# # Smaller datasets for testing
# train_dataset = train_d_raw.select(range(TEST_DATASET_SIZE))
# eval_dataset = test_raw.select(range(TEST_DATASET_SIZE))

print("Train dataset positive: {}/{}".format(np.sum(train_dataset['label']), train_dataset.num_rows))
print("Eval dataset positive: {}/{}".format(np.sum(eval_dataset['label']), eval_dataset.num_rows))

Train dataset positive: 6233/12500
Eval dataset positive: 12500/25000


In [9]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=TOKEN_MAX_LENGTH)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [10]:
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True)

  0%|          | 0/13 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

In [11]:
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["text"])
tokenized_train_dataset = tokenized_train_dataset.rename_column("label", "labels")
tokenized_train_dataset.set_format("torch")

tokenized_eval_dataset = tokenized_eval_dataset.remove_columns(["text"])
tokenized_eval_dataset = tokenized_eval_dataset.rename_column("label", "labels")
tokenized_eval_dataset.set_format("torch")

In [12]:
# TODO: Consider whether masking is needed....

In [13]:
train_dataloader = DataLoader(tokenized_train_dataset, shuffle=True, batch_size=BATCH_SIZE)
eval_dataloader = DataLoader(tokenized_eval_dataset, batch_size=BATCH_SIZE)

<a id='trainer'></a>

# BERT Training

## Prepare training settings

In [14]:
# Using "output_hidden_states = True" caused a CUDA out of memory error. Maybe can't use it while training, only during evaluation.

# model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2, output_hidden_states = True)
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [15]:
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)

In [16]:
num_training_steps = EPOCHS * len(train_dataloader)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer, 
    num_warmup_steps=0, 
    num_training_steps=num_training_steps
)

In [17]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device);

## Train Model

In [18]:
metric = evaluate.load("accuracy")
def evaluate_model(e_model, e_dataloader):
  e_progress_bar = tqdm(range(len(e_dataloader)))
  e_model.eval()
  e_total_loss = 0
  e_count = 0
  for batch in e_dataloader:
      batch = {k: v.to(device) for k, v in batch.items()}
      with torch.no_grad():
          outputs = e_model(**batch)

      e_logits = outputs.logits
      e_count += batch['labels'].shape[0]
      e_total_loss += outputs.loss.detach() * batch['labels'].shape[0]
      predictions = torch.argmax(e_logits, dim=-1)
      metric.add_batch(predictions=predictions, references=batch["labels"])
      e_progress_bar.update(1)

  e_acc = metric.compute()['accuracy']
  e_loss = e_total_loss / e_count
  return e_acc, e_loss

In [19]:
for epoch in range(EPOCHS):
    print("Epoch {}".format(epoch))
    progress_bar = tqdm(range(len(train_dataloader)))
    model.train()
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
      
    if EVALUATE_AFTER_EPOCH:
        acc, loss = evaluate_model(model, eval_dataloader)
        print("  Accuracy: {}".format(acc))
        print("  Loss: {}".format(loss))

Epoch 0


  0%|          | 0/1563 [00:00<?, ?it/s]

  0%|          | 0/3125 [00:00<?, ?it/s]

  Accuracy: 0.90464
  Loss: 0.24024541676044464
Epoch 1


  0%|          | 0/1563 [00:00<?, ?it/s]

  0%|          | 0/3125 [00:00<?, ?it/s]

  Accuracy: 0.90344
  Loss: 0.2785855531692505
Epoch 2


  0%|          | 0/1563 [00:00<?, ?it/s]

  0%|          | 0/3125 [00:00<?, ?it/s]

  Accuracy: 0.90688
  Loss: 0.33120912313461304


In [20]:
print(evaluate_model(model, eval_dataloader))

  0%|          | 0/3125 [00:00<?, ?it/s]

(0.90688, tensor(0.3312, device='cuda:0'))


## Save model weights

In [21]:
model.save_pretrained("bert_imdb_base")

In [22]:
!zip -r bert_imdb_clean_model.zip bert_imdb_base

  adding: bert_imdb_base/ (stored 0%)
  adding: bert_imdb_base/pytorch_model.bin (deflated 7%)
  adding: bert_imdb_base/config.json (deflated 49%)


<a id='keras'></a>